In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from sklearn.model_selection import train_test_split

In [ ]:
# --- 1. 데이터 로드 및 준비 ---
# 전처리 완료된 데이터 삽입
df = pd.read_csv('adolescent_health_data.csv')


# --- 2. Prefix 정의 및 칼럼 그룹핑 ---
# 이미지에 나온 prefix들을 리스트로 정의합니다.
# 'M'은 타겟이므로 제외합니다.
prefixes = [
    'TC_', 'AC_', 'WC_', 'PA_', 'F_', 'I_', 'V_', 'S_', 'O_',
    'HW_', 'AS_', 'RH_', 'ECZ_', 'INT_', 'DR_', 'E_'
]

# 타겟 칼럼(M으로 시작)과 피처 칼럼을 분리합니다.
m_targets = [col for col in df.columns if col.startswith('M_')]
all_features = [col for col in df.columns if col not in m_targets]

# Prefix별로 피처 칼럼을 그룹핑합니다.
feature_groups = {}
no_prefix_features = []

for feature in all_features:
    found_prefix = False
    for prefix in prefixes:
        if feature.startswith(prefix):
            if prefix not in feature_groups:
                feature_groups[prefix] = []
            feature_groups[prefix].append(feature)
            found_prefix = True
            break
    if not found_prefix:
        # 어떤 prefix에도 속하지 않는 칼럼들 (예: 'user_id', 'age' 등)
        no_prefix_features.append(feature)

# Prefix 없는 피처들도 하나의 그룹으로 추가
if no_prefix_features:
    feature_groups['NO_PREFIX_'] = no_prefix_features

print("✅ 타겟 (M_):", m_targets)
print("✅ 그룹핑된 피처:")
for prefix, cols in feature_groups.items():
    print(f"  - {prefix}: {cols}")




# --- 3. 각 타겟에 대해 개별 모델 학습 (반복문) ---
history_dict = {} # 모델별 학습 기록 저장

for target_col in m_targets:
    print(f"\n{'='*50}")
    print(f"🚀 모델 학습 시작: Target = {target_col}")
    print(f"{'='*50}")

    # 데이터 분리 (매번 동일하게 분리하기 위해 random_state 고정)
    X = df[all_features]
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- Keras Functional API를 사용한 모델 구축 ---
    # 1단계: Prefix 그룹별 입력 및 1차 처리 레이어
    input_layers = []
    processed_outputs = []

    for prefix, cols in feature_groups.items():
        # 각 그룹에 맞는 입력층 생성
        group_input = Input(shape=(len(cols),), name=f'input_{prefix}')
        input_layers.append(group_input)
        
        # 각 그룹을 하나의 노드로 압축하는 Dense 레이어
        # 그룹의 특성을 요약하는 역할을 합니다.
        group_output = Dense(1, activation='relu', name=f'processed_{prefix}')(group_input)
        processed_outputs.append(group_output)

    # 2단계: 1차 처리된 노드들 결합
    # 모든 그룹의 요약 노드들을 하나로 합칩니다.
    concatenated = Concatenate()(processed_outputs)

    # (선택) 중간에 은닉층(Hidden Layer) 추가
    # 더 복잡한 관계를 학습하고 싶을 때 추가합니다. 16개 노드 사용.
    hidden_layer = Dense(16, activation='relu', name='hidden_layer_1')(concatenated)

    # 3단계: 최종 출력 레이어
    # 타겟 변수를 예측합니다. (이진 분류로 가정하여 sigmoid 사용)
    output_layer = Dense(1, activation='sigmoid', name='output_layer')(hidden_layer)
    
    # 모델 정의
    # feature_groups의 순서에 맞게 실제 데이터를 넣어주기 위해 입력 데이터를 리스트로 준비
    model_inputs = [X_train[cols] for prefix, cols in feature_groups.items()]
    test_inputs = [X_test[cols] for prefix, cols in feature_groups.items()]

    model = Model(inputs=list(input_layers), outputs=output_layer)

    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='binary_crossentropy', # 타겟이 0 또는 1일 경우
                  metrics=['accuracy'])
    
    model.summary()

    # 모델 학습
    history = model.fit(
        model_inputs,
        y_train,
        validation_data=(test_inputs, y_test),
        epochs=10, # 실제 데이터에 맞게 조절
        batch_size=32,
        verbose=0 # 학습 과정 로그 생략 (1로 바꾸면 보임)
    )

    history_dict[target_col] = history
    print(f"✅ 모델 학습 완료: {target_col}")
    
    # 모델 평가
    loss, accuracy = model.evaluate(test_inputs, y_test)
    print(f"📈 테스트 정확도: {accuracy:.4f}")

In [ ]:
# --- (1, 2번 코드는 위와 동일) ---

# --- 3. 다중 출력 모델 구축 및 학습 ---
history_dict = {} # 모델별 학습 기록 저장

for target_col in m_targets:
    print(f"\n{'='*50}")
    print(f"🚀 모델 학습 시작: Target = {target_col}")
    print(f"{'='*50}")

# 데이터 분리
X = df[all_features]
y = df[m_targets] # y를 모든 M_ 칼럼으로 지정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Keras Functional API를 사용한 모델 구축 ---
# 1단계: Prefix 그룹별 입력 및 1차 처리 레이어
input_layers = []
processed_outputs = []

for prefix, cols in feature_groups.items():
    group_input = Input(shape=(len(cols),), name=f'input_{prefix}')
    input_layers.append(group_input)
    group_output = Dense(1, activation='relu', name=f'processed_{prefix}')(group_input)
    processed_outputs.append(group_output)

# 2단계: 1차 처리된 노드들 결합
concatenated = Concatenate()(processed_outputs)
hidden_layer = Dense(16, activation='relu', name='hidden_layer_1')(concatenated)

# 3단계: 최종 출력 레이어
# 출력 노드 수를 타겟 칼럼의 개수(len(m_targets))로 설정합니다.
output_layer = Dense(len(m_targets), activation='sigmoid', name='output_layer')(hidden_layer)

# 모델 정의
model_inputs = [X_train[cols] for prefix, cols in feature_groups.items()]
test_inputs = [X_test[cols] for prefix, cols in feature_groups.items()]

multi_output_model = Model(inputs=list(input_layers), outputs=output_layer)

# 모델 컴파일
multi_output_model.compile(optimizer='adam',
                           loss='binary_crossentropy', # 모든 출력이 이진 분류일 경우
                           metrics=['accuracy'])

multi_output_model.summary()

# 모델 학습
print("\n🚀 다중 출력 모델 학습 시작...")
history = multi_output_model.fit(
    model_inputs,
    y_train,
    validation_data=(test_inputs, y_test),
    epochs=10,
    batch_size=32
)
print("✅ 다중 출력 모델 학습 완료!")

# 모델 평가
loss, accuracy = multi_output_model.evaluate(test_inputs, y_test)
print(f"📈 테스트 정확도: {accuracy:.4f}")